In [5]:
import pandas as pd
import requests
import time

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
TICKERS_URL = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
LIMIT = 1  # number of past reports (1 = latest)
SLEEP_TIME = 1  # avoid rate limit

# === LOAD TICKERS ===
print("Loading S&P 500 tickers...")
tickers_df = pd.read_csv(TICKERS_URL)
tickers = tickers_df['Symbol'].tolist()

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit={LIMIT}&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]
    return {}

# === MAIN LOOP ===
all_data = []

print("Fetching data for each ticker...")
for i, symbol in enumerate(tickers[0:50]):
    print(f"[{i+1}/{len(tickers)}] {symbol}")

    income = get_fmp_data("income-statement", symbol)
    balance = get_fmp_data("balance-sheet-statement", symbol)
    cashflow = get_fmp_data("cash-flow-statement", symbol)

    if income or balance or cashflow:
        merged = {
            "symbol": symbol,
            "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
            **income,
            **{f"balance_{k}": v for k, v in balance.items()},
            **{f"cashflow_{k}": v for k, v in cashflow.items()}
        }
        all_data.append(merged)

    time.sleep(SLEEP_TIME)

# === SAVE RESULTS ===
df_0_50 = pd.DataFrame(all_data)
df_0_50.to_csv("0_50_sp500_fundamentals.csv", index=False)
print("✅ Data saved to '0_50_sp500_fundamentals.csv'")


Loading S&P 500 tickers...
Fetching data for each ticker...
[1/503] MMM
[2/503] AOS
[3/503] ABT
[4/503] ABBV
[5/503] ACN
[6/503] ADBE
[7/503] AMD
[8/503] AES
[9/503] AFL
[10/503] A
[11/503] APD
[12/503] ABNB
[13/503] AKAM
[14/503] ALB
[15/503] ARE
[16/503] ALGN
[17/503] ALLE
[18/503] LNT
[19/503] ALL
[20/503] GOOGL
[21/503] GOOG
[22/503] MO
[23/503] AMZN
[24/503] AMCR
[25/503] AEE
[26/503] AEP
[27/503] AXP
[28/503] AIG
[29/503] AMT
[30/503] AWK
[31/503] AMP
[32/503] AME
[33/503] AMGN
[34/503] APH
[35/503] ADI
[36/503] ANSS
[37/503] AON
[38/503] APA
[39/503] APO
[40/503] AAPL
[41/503] AMAT
[42/503] APTV
[43/503] ACGL
[44/503] ADM
[45/503] ANET
[46/503] AJG
[47/503] AIZ
[48/503] T
[49/503] ATO
[50/503] ADSK
✅ Data saved to '0_50_sp500_fundamentals.csv'


In [11]:
df_0_50.shape

(50, 133)

In [12]:
df_0_50.head()

,symbol,reportDate,date,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,revenue,...,cashflow_netCashUsedProvidedByFinancingActivities,cashflow_effectOfForexChangesOnCash,cashflow_netChangeInCash,cashflow_cashAtEndOfPeriod,cashflow_cashAtBeginningOfPeriod,cashflow_operatingCashFlow,cashflow_capitalExpenditure,cashflow_freeCashFlow,cashflow_link,cashflow_finalLink
0,MMM,2024-12-31,2024-12-31,USD,0000066740,2025-02-05,2025-02-05 09:57:15,2024,FY,24575000000,...,1098000000,-44000000.0,-333000000,5600000000,5933000000,1819000000,-1181000000,638000000,https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...
1,AOS,2024-12-31,2024-12-31,USD,0000091142,2025-02-11,2025-02-11 17:28:15,2024,FY,3818100000,...,-408400000,-6600000.0,-100300000,239600000,339900000,581800000,-108000000,473800000,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...
2,ABT,2024-12-31,2024-12-31,USD,0000001800,2025-02-21,2025-02-21 16:07:20,2024,FY,41950000000,...,-5404000000,-96000000.0,720000000,7616000000,6896000000,8558000000,-2207000000,6351000000,https://www.sec.gov/Archives/edgar/data/1800/0...,https://www.sec.gov/Archives/edgar/data/1800/0...
3,ABBV,2024-12-31,2024-12-31,USD,0001551152,2025-02-14,2025-02-14 13:17:23,2024,FY,56334000000,...,-5211000000,-65000000.0,-7290000000,5524000000,12814000000,18806000000,0,18806000000,https://www.sec.gov/Archives/edgar/data/155115...,https://www.sec.gov/Archives/edgar/data/155115...
4,ACN,2024-08-31,2024-08-31,USD,0001467373,2024-10-10,2024-10-10 06:43:59,2024,FY,64896464000,...,-6063508000,-46264000.0,-4040563000,5004469000,9045032000,9131027000,-516509000,8614518000,https://www.sec.gov/Archives/edgar/data/146737...,https://www.sec.gov/Archives/edgar/data/146737...


Ho runnato solo i primi 50, poi vanno mergiati tutti insieme

# -------- PROVA CON SOLO TICKER APPLE --------

In [13]:
import pandas as pd
import requests

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
symbol = "AAPL"

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit=1&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]  # solo l'ultimo anno
    return {}

# === FETCH DATA ===
print(f"Fetching annual fundamentals for {symbol}")

income = get_fmp_data("income-statement", symbol)
balance = get_fmp_data("balance-sheet-statement", symbol)
cashflow = get_fmp_data("cash-flow-statement", symbol)

# === MERGE & CREATE DATAFRAME ===
merged = {
    "symbol": symbol,
    "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
    **income,
    **{f"balance_{k}": v for k, v in balance.items()},
    **{f"cashflow_{k}": v for k, v in cashflow.items()}
}

df_apple = pd.DataFrame([merged])  # salva in DataFrame con una riga
print(df_apple.head())  # anteprima



Fetching annual fundamentals for AAPL
  symbol  reportDate        date reportedCurrency         cik fillingDate  \
0   AAPL  2024-09-28  2024-09-28              USD  0000320193  2024-11-01   

          acceptedDate calendarYear period       revenue  ...  \
0  2024-11-01 06:01:36         2024     FY  391035000000  ...   

   cashflow_netCashUsedProvidedByFinancingActivities  \
0                                      -121983000000   

   cashflow_effectOfForexChangesOnCash  cashflow_netChangeInCash  \
0                                    0                -794000000   

   cashflow_cashAtEndOfPeriod  cashflow_cashAtBeginningOfPeriod  \
0                 29943000000                       30737000000   

   cashflow_operatingCashFlow  cashflow_capitalExpenditure  \
0                118254000000                  -9447000000   

   cashflow_freeCashFlow                                      cashflow_link  \
0           108807000000  https://www.sec.gov/Archives/edgar/data/320193...   

       

In [14]:
df_apple.columns

Index(['symbol', 'reportDate', 'date', 'reportedCurrency', 'cik',
       'fillingDate', 'acceptedDate', 'calendarYear', 'period', 'revenue',
       ...
       'cashflow_netCashUsedProvidedByFinancingActivities',
       'cashflow_effectOfForexChangesOnCash', 'cashflow_netChangeInCash',
       'cashflow_cashAtEndOfPeriod', 'cashflow_cashAtBeginningOfPeriod',
       'cashflow_operatingCashFlow', 'cashflow_capitalExpenditure',
       'cashflow_freeCashFlow', 'cashflow_link', 'cashflow_finalLink'],
      dtype='object', length=133)

## ---------- VARIABILI IMPORTANTI ED ESEMPI ------------

### VARIABILI IMPORTANTI PER INCOME

Variabile | Descrizione | Frequenza reale di variazione

revenue | Ricavi totali della società | Trimestrale (earnings)

grossProfit | Ricavi - costo del venduto | Trimestrale

operatingIncome | Risultato operativo prima delle tasse e interessi | Trimestrale

netIncome | Utile netto | Trimestrale

eps (Earnings per Share) | Utile per azione | Trimestrale

interestExpense | Interessi passivi | Trimestrale

In [ ]:
# Esempio di output per income

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "revenue": 391035000000,
        "costOfRevenue": 210352000000,
        "grossProfit": 180683000000,
        "grossProfitRatio": 0.4620634982,
        "researchAndDevelopmentExpenses": 31370000000,
        "generalAndAdministrativeExpenses": 0,
        "sellingAndMarketingExpenses": 0,
        "sellingGeneralAndAdministrativeExpenses": 26097000000,
        "otherExpenses": 0,
        "operatingExpenses": 57467000000,
        "costAndExpenses": 267819000000,
        "interestIncome": 0,
        "interestExpense": 0,
        "depreciationAndAmortization": 11445000000,
        "ebitda": 134661000000,
        "ebitdaratio": 0.3443707085,
        "operatingIncome": 123216000000,
        "operatingIncomeRatio": 0.3151022287,
        "totalOtherIncomeExpensesNet": 269000000,
        "incomeBeforeTax": 123485000000,
        "incomeBeforeTaxRatio": 0.3157901467,
        "incomeTaxExpense": 29749000000,
        "netIncome": 93736000000,
        "netIncomeRatio": 0.2397125577,
        "eps": 6.11,
        "epsdiluted": 6.08,
        "weightedAverageShsOut": 15343783000,
        "weightedAverageShsOutDil": 15408095000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER BALANCE

Variabile | Descrizione | Frequenza reale

totalAssets | Totale attivo | Trimestrale

totalLiabilities | Totale passivo | Trimestrale

cashAndCashEquivalents | Cassa disponibile | Trimestrale

totalDebt | Totale debito (a breve e lungo termine) | Trimestrale

inventory | Scorte | Trimestrale

shareholderEquity | Capitale proprio | Trimestrale

In [ ]:
# Esempio di output per balance

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "cashAndCashEquivalents": 29943000000,
        "shortTermInvestments": 35228000000,
        "cashAndShortTermInvestments": 65171000000,
        "netReceivables": 66243000000,
        "inventory": 7286000000,
        "otherCurrentAssets": 14287000000,
        "totalCurrentAssets": 152987000000,
        "propertyPlantEquipmentNet": 45680000000,
        "goodwill": 0,
        "intangibleAssets": 0,
        "goodwillAndIntangibleAssets": 0,
        "longTermInvestments": 91479000000,
        "taxAssets": 19499000000,
        "otherNonCurrentAssets": 55335000000,
        "totalNonCurrentAssets": 211993000000,
        "otherAssets": 0,
        "totalAssets": 364980000000,
        "accountPayables": 68960000000,
        "shortTermDebt": 22511000000,
        "taxPayables": 26601000000,
        "deferredRevenue": 8249000000,
        "otherCurrentLiabilities": 50071000000,
        "totalCurrentLiabilities": 176392000000,
        "longTermDebt": 96548000000,
        "deferredRevenueNonCurrent": 0,
        "deferredTaxLiabilitiesNonCurrent": 0,
        "otherNonCurrentLiabilities": 35090000000,
        "totalNonCurrentLiabilities": 131638000000,
        "otherLiabilities": 0,
        "capitalLeaseObligations": 12430000000,
        "totalLiabilities": 308030000000,
        "preferredStock": 0,
        "commonStock": 83276000000,
        "retainedEarnings": -19154000000,
        "accumulatedOtherComprehensiveIncomeLoss": -7172000000,
        "othertotalStockholdersEquity": 0,
        "totalStockholdersEquity": 56950000000,
        "totalEquity": 56950000000,
        "totalLiabilitiesAndStockholdersEquity": 364980000000,
        "minorityInterest": 0,
        "totalLiabilitiesAndTotalEquity": 364980000000,
        "totalInvestments": 126707000000,
        "totalDebt": 106629000000,
        "netDebt": 76686000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER CASHFLOW

Variabile | Descrizione | Frequenza reale

operatingCashFlow | Flussi di cassa dalle operazioni | Trimestrale

capitalExpenditure | Investimenti in capitale (CAPEX) | Trimestrale

freeCashFlow | Cash flow operativo - CAPEX | Trimestrale

dividendsPaid | Dividendi distribuiti | Trimestrale

In [ ]:
# Esempio di output per cashflow
[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "netIncome": 93736000000,
        "depreciationAndAmortization": 11445000000,
        "deferredIncomeTax": 0,
        "stockBasedCompensation": 11688000000,
        "changeInWorkingCapital": 3651000000,
        "accountsReceivables": -5144000000,
        "inventory": -1046000000,
        "accountsPayables": 6020000000,
        "otherWorkingCapital": 3821000000,
        "otherNonCashItems": -2266000000,
        "netCashProvidedByOperatingActivities": 118254000000,
        "investmentsInPropertyPlantAndEquipment": -9447000000,
        "acquisitionsNet": 0,
        "purchasesOfInvestments": -48656000000,
        "salesMaturitiesOfInvestments": 62346000000,
        "otherInvestingActivites": -1308000000,
        "netCashUsedForInvestingActivites": 2935000000,
        "debtRepayment": -5998000000,
        "commonStockIssued": 0,
        "commonStockRepurchased": -94949000000,
        "dividendsPaid": -15234000000,
        "otherFinancingActivites": -5802000000,
        "netCashUsedProvidedByFinancingActivities": -121983000000,
        "effectOfForexChangesOnCash": 0,
        "netChangeInCash": -794000000,
        "cashAtEndOfPeriod": 29943000000,
        "cashAtBeginningOfPeriod": 30737000000,
        "operatingCashFlow": 118254000000,
        "capitalExpenditure": -9447000000,
        "freeCashFlow": 108807000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]